<a href="https://colab.research.google.com/github/yingzibu/JAK_ML/blob/main/examples/expeirments/finetune/ADMET_10_30_ST_scale_cetain_metrics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import torch
print(torch.__version__)
print('cuda: ', torch.cuda.is_available())

2.1.0+cu118
cuda:  True


In [ ]:
! pip install rdkit --quiet
! pip install PyTDC --quiet
! pip install mycolorpy --quiet
! pip install selfies  --quiet
! pip install pubchempy --quiet
! pip install dgllife --quiet
! pip install molvs --quiet
! pip install dgl -f https://data.dgl.ai/wheels/cu118/repo.html --quiet
! pip install dglgo -f https://data.dgl.ai/wheels-test/repo.html --quiet

! pip install DeepPurpose --quiet
! pip install git+https://github.com/bp-kelley/descriptastorus --quiet
! pip install pandas-flavor --quiet

#### Code

In [5]:
from os import walk
import os
file_types = ['bin', 'pth']

# clean certain type of file in path
def clean_files(path='/content/drive/MyDrive/ADMET/', file_types = ['pth']):
    files = next(walk(path), (None, None, []))[2]
    for file in files:
        if isinstance(file, str):
            file_type = file.split('.')[-1]
            if file_type in file_types:
                os.remove(file); print(f'{file} removed from {path}')

In [6]:
cd /content/drive/MyDrive/ADMET

/content/drive/MyDrive/ADMET


In [7]:
# test scripts.func_utils.py

from scripts.func_utils import make_path, convert_with_qed_sa, get_min, \
                                plot_loss, plot_performance

from scripts.eval_utils import *
from scripts.preprocess_mols import *
from scripts.model_architecture import *
from scripts.dataset import *
from scripts.train import *
from tdc.single_pred import ADME
from tdc.single_pred import Tox
from tdc.utils import retrieve_label_name_list
import pandas as pd

label_list = retrieve_label_name_list('herg_central')

def collect_data_10_27(names:list, clean_mol_=False, verbose=False):
    if isinstance(names, str): names = [names]
    name_adme = ['Caco2_Wang', 'Lipophilicity_AstraZeneca',
                 'HydrationFreeEnergy_FreeSolv',
                 'Solubility_AqSolDB'] # regression task
    name_adme+= ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
                'CYP1A2_Veith', 'CYP2C9_Veith'] + \
                ['BBB_Martins', 'Bioavailability_Ma', 'Pgp_Broccatelli',
                 'HIA_Hou','PAMPA_NCATS'] # classify
    print('collect data for: ', names)
    label_list = retrieve_label_name_list('herg_central')
    for i, name in enumerate(names):
        if verbose: print('*'*15, name, '*'*15)
        if name in label_list: data = Tox(name='herg_central', label_name=name)
        elif name in name_adme: data = ADME(name=name)
        else:
            try: data = Tox(name=name)
            except: print('cannot read data!'); return
            if verbose: data.label_distribution()
            # data.label_distribution()
        split = data.get_split()
        train, valid, test = split['train'], split['valid'], split['test']
        if clean_mol_:
            train, valid, test = clean_mol(train), clean_mol(valid), clean_mol(test)

        train = rename_cols(train[['Drug', 'Y']], name)
        valid = rename_cols(valid[['Drug', 'Y']], name)
        test  = rename_cols(test[['Drug', 'Y']],  name)

        if i == 0: trains, valids, tests = train.copy(), valid.copy(), test.copy()
        else:
            trains = trains.merge(train, how='outer')
            valids = valids.merge(valid, how='outer')
            tests = tests.merge(test, how='outer')

    return trains, valids, tests
# trains, valids, tests = collect_data_10_27(names_reg[0])

from scripts.get_vocab import *

def get_multi_loader(trains, valids, tests, config):
    names = config['prop_names']
    vocab = None if 'vocab' not in config else config['vocab']
    batch_size = config['batch_size']
    model_type = config['model_type']

    print('---> loader for', names)
    params_ = {'batch_size': batch_size, 'shuffle': True,
               'drop_last': False, 'num_workers': 0}
    param_t = {'batch_size': batch_size, 'shuffle': False,
               'drop_last': False, 'num_workers': 0}

    # NEED TO CHANGE HERE TO INCLUDE SELFIES
    if model_type == 'RNN'and vocab == None:
        df = pd.concat([trains, valids, tests], ignore_index=True, axis=0)
        vocab = get_vocab(df)
    train_loader = get_loader(trains, names, params_, model_type, vocab)
    valid_loader = get_loader(valids, names, params_, model_type, vocab)
    test_loader  = get_loader(tests,  names, param_t, model_type, vocab)
    return train_loader, valid_loader, test_loader, vocab


VOCAB_TYPE: smiles, could change from ['char', 'smiles', 'selfies'] at get_vocab.py
Setting the default backend to "pytorch". You can change it in the ~/.dgl/config.json file or export the DGLBACKEND environment variable.  Valid options are: pytorch, mxnet, tensorflow (all lowercase)


DGL backend not selected or invalid.  Assuming PyTorch for now.


In [61]:

import yaml
def yml_report(yml_path, recalculate=False, ver=False):
    """
    given yml path or yml data, return the test set performance
    param
        yml_path        : str,  the path of yml file
        recalculate     : bool, if true, will calculate from scratch
        ver             : bool, if true, will print detailed configs
    return perfm_dict   : dict, contain performance and test loss
    """
    if isinstance(yml_path, str): # the path string was input
        with open(yml_path, 'r') as f: data = yaml.safe_load(f)
    elif isinstance(yml_path, dict):   data = yml_path # data was input

    config = data['config']
    model_type = config['model_type']
    task_names = config['prop_names']
    perfm_dict =  data['performance']

    if len(perfm_dict) != 0 and recalculate == False:
        # during training, has evaluted test set, no need to calculate again
        # However for regression, the pred vs true value for test is not saved
        # if need the regression pred vs true value graph, need recalculate
        if ver: # print model config, and the training saved info
            print('#'*68); print('#'*30, 'CONFIG', '#'*30); print('#'*68)
            for i, j in config.items(): print(i, ':', j)
            print('#'*68)
            print('Model parameters: ', data['params_num'])
            times_list = data['times_list']
            print(f'Train time: {np.mean(times_list):.3f}'
                  f'+/-{np.std(times_list):.3f} ms')
            print(f"best epoch: {data['best_epoch']}, ",
                  f"min loss: {data['min_loss']:.4f}")
            plot_loss(data['train_dict'], data['valid_dict'], name='valid',
                      title_name= f'loss during training {model_type}')

    else: # recalculate from scratch using test data
        vocab = None if 'vocab' not in config else config['vocab']
        trains, valids, tests = collect_data_10_27(task_names)
        if config['scale_dict'] != None: # scale is done
            trains, valids, tests, dict_scale = scale(trains, valids, tests)
            assert config['scale_dict'] == dict_scale
        batch_size = config['batch_size']
        param_t = {'batch_size': batch_size, 'shuffle': False,
                'drop_last': False, 'num_workers': 0}
        test_loader = get_loader(tests, task_names, param_t, model_type, vocab)
        models = PRED(**config); models.load_status(data)
        outputs = models.eval(test_loader, ver=ver)
        perfm_dict = outputs[0]
    return perfm_dict


import numpy as np

#                0      1        2       3        4     5      6      7      8
cls_metrics = ['acc', 'w_acc', 'prec', 'recall', 'sp', 'f1', 'auc', 'mcc', 'ap']
reg_metrics = ['mae', 'mse', 'rmse', 'r2']
d = {'reg': [0,   2,    3], 'cls': [0,   5,  6]}
#            mae, rmse, r2          acc, f1, auc


# evaluate performance list, if directly from yml_file saved performance (dict)
# need to [yml_file_data['performance']] to convert dict into list
def eval_perf_list(perfs:list, name:list,
                   metrics_dict=d, # could be {} # eval all
                   reg_metrics_all=reg_metrics,
                   cls_metrics_all=cls_metrics):
    """
    The same model type for multiple times, performance saved in list perfs
    Aim: evaluate performance of name, calculate mean and std for multiple run
    : param metrics_dict: dict, if None, print all metrics
                          example: {'reg': [0, 2, 3], 'cls': [0, 5, 6]}
    """
    if len(metrics_dict) == 0:  # will print all metrics
        metrics_dict['reg'] = [i for i in range(len(reg_metrics_all))]
        metrics_dict['cls'] = [i for i in range(len(cls_metrics_all))]
    if isinstance(name, str): name = [name]
    if isinstance(perfs, dict): perfs = [perfs]
    repeat_time = len(perfs) # the same model was run for # repeat_time times

    if repeat_time > 1: # multiple run, find the lowest loss
        loss_list = [p['loss'] for p in perfs]
        best_model_idx = np.argmin(loss_list) # has the lowest loss
        best_perf = perfs[best_model_idx]
        print('repeated num #', repeat_time, end=" ")
        print(f'idx {best_model_idx} has the lowest loss from {loss_list}')
    else: best_perf = None
    for n in name:
        IS_R = names_dict[n]
        # print(n, '\tRegression?', IS_R);
        if IS_R:
              idxs = metrics_dict['reg']; ms=[reg_metrics_all[i] for i in idxs]
        else: idxs = metrics_dict['cls']; ms=[cls_metrics_all[i] for i in idxs]

        results = {}
        for idx, i in enumerate(perfs): # access idx_th evaluation in perfs
            r = i[n]; results[idx] = r  # collect the evaluation for name n

        means, stds = [], []

        for idx_v in range(len(r)):
            cur_values = []
            for idx in range(repeat_time):
                cur_v = results[idx][idx_v]; cur_values.append(cur_v)
            mean_here, std_here = np.mean(cur_values), np.std(cur_values)
            means.append(mean_here); stds.append(std_here)
            # print(f'{ms[idx_v]}\t: {mean_here:.3f} +/- {std_here:.3f}')

        print('*'*20, n, '*'*20,  end=' \n\t')
        for k in ms: print('\t', k, end = ' \t ')
        print()

        for idx_final, (i, j) in enumerate(zip(means, stds)):
            if idx_final == 0:    print(end='\t| ')
            if idx_final in idxs: print(f'{i:.3f} +/- {j:.3f}', end=' | ')
        if best_perf != None:
            print(f'\n idx {best_model_idx}: ', end='| ')
            for i in idxs:
                print(f'{best_perf[n][i]:.3f} +/- {0:.3f}', end=' | ')
        print('\n')

        # break


#### Constant and configs

In [8]:
### CONSTANTS ###
names_reg = ['Caco2_Wang', 'Lipophilicity_AstraZeneca',
         'HydrationFreeEnergy_FreeSolv', 'Solubility_AqSolDB', 'LD50_Zhu'] # regression task
names_cls = ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
             'CYP1A2_Veith', 'CYP2C9_Veith'] + \
            ['BBB_Martins', 'Bioavailability_Ma',
             'Pgp_Broccatelli', 'HIA_Hou','PAMPA_NCATS'] + \
            ['hERG_Karim', 'AMES']

names_dict = {}
for name in names_reg + names_cls:
    if name in names_reg:   names_dict[name] = True  # regression task
    elif name in names_cls: names_dict[name] = False # classification task
names_all = list(names_dict.keys())

model_types = ['MLP', 'AttentiveFP', 'GIN', 'RNN']

#                0      1        2       3        4     5      6      7      8
cls_metrics = ['acc', 'w_acc', 'prec', 'recall', 'sp', 'f1', 'auc', 'mcc', 'ap']
reg_metrics = ['mae', 'mse', 'rmse', 'r2']


In [ ]:
in_dim = 256
hid_dims = [128, 64, 16]
dropout = 0.5
lr = 3e-4
wd = 1e-5
MAX_EPOCH = 1000
patience = 30           # stop if loss no decrease after epochs # patience
verbose_frequency = 100 # print evaluation every # verbose_frequency epoch
batch_size = 128

# special for AttentiveFP
graph_feat_size = 300
n_layers = 5
num_timesteps = 1   # times of updating the graph representations with GRU

# special for GIN: pretrain model types for selection:
pre_models_GIN = ['gin_supervised_contextpred', 'gin_supervised_infomax',
                     'gin_supervised_edgepred', 'gin_supervised_masking']
pre_model_num = 0    # choose from pre_models for GIN


# if VOCAB_TYPE == 'smiles':
import yaml
def load_vocab(VOCAB_TYPE):
    try:
        with open(f'vocab/{VOCAB_TYPE}.yml', 'r') as f: data = yaml.safe_load(f)
        vocab = data['vocab']; assert VOCAB_TYPE == data['vocab_type']
    except: vocab = None
    return vocab

# special for RNN:
VOCAB = load_vocab(VOCAB_TYPE)
Bid = True
GRU_num_layers = 3
GRU_dim = 256



scale_dict = None

def get_config(model_type, names,
               pre_model_num=pre_model_num, scale_dict=scale_dict):
    """
    Get config to initialize model
        param model_type: str, ['MLP', 'AttentiveFP', 'GIN', 'RNN']
        param names: list, task names
        param scale_dict: dict,
            if the task is regression, could scale label values
                            {name: [value_min, value_max], ...}
        param pre_model_num: int, [0, 1, 2, 3]
            if model_type is 'GIN', 4 types of pretrained models to choose from
    Returns config that could be used as PRED(**config)
    """
    pre_models_GIN = ['gin_supervised_contextpred', 'gin_supervised_infomax',
                         'gin_supervised_edgepred', 'gin_supervised_masking']

    # print(scale_dict)
    if isinstance(names, str): names = [names]
    IS_R = [names_dict[name] for name in names]
    config_MLP = {'model_type': 'MLP',
            'in_dim': 167,
            'hid_dims': hid_dims,
            'out_dim': len(names),
            'prop_names': names,
            'dropout': dropout,
            'IS_R': IS_R,
            'batch_size': batch_size,
            'lr': lr,
            'wd': wd,
            'patience': patience,
            'verbose_freq': verbose_frequency,
            'model_path': f'ckpt_MLP.pt',
            'scale_dict': scale_dict}

    config_ATF = {'model_type': 'AttentiveFP',
            'graph_feat_size': graph_feat_size,
            'num_timesteps': num_timesteps,
            'n_layers': n_layers,
            'out_dim': len(names),
            'prop_names': names,
            'dropout': dropout,
            'IS_R': IS_R,
            'batch_size': batch_size,
            'lr': lr,
            'wd': wd,
            'patience': patience,
            'verbose_freq': verbose_frequency,
            'model_path': 'ckpt_AT.pt',
            'scale_dict': scale_dict}

    config_GIN = {'model_type': 'GIN',
            'pretrain_model': pre_models_GIN[pre_model_num],
            'in_dim': in_dim,
            'hid_dims': hid_dims,
            'out_dim': len(names),
            'prop_names': names,
            'dropout': dropout,
            'batch_size': batch_size,
            'IS_R': IS_R,
            'lr': lr,
            'wd': wd,
            'patience': patience,
            'verbose_freq': verbose_frequency,
            'model_path': f'ckpt_GIN_{pre_models_GIN[pre_model_num]}.pt',
            'scale_dict': scale_dict}

    config_RNN = {'model_type': 'RNN',
              'vocab': VOCAB,
              'vocab_type': VOCAB_TYPE,
              'Bidirect': Bid,
              'num_layers': GRU_num_layers,
              'GRU_dim': GRU_dim,
              'out_dim': len(names),
              'prop_names': names,
              'dropout': dropout,
              'IS_R': IS_R,
              'device': 'cuda',
              'batch_size': batch_size,
              'lr': lr,
              'wd': wd,
              'patience': patience,
              'verbose_freq': verbose_frequency,
              'model_path': f'ckpt_RNN_{VOCAB_TYPE}.pt',
              'scale_dict': scale_dict}

    if model_type == 'MLP':           con_MO = config_MLP
    elif model_type == 'AttentiveFP': con_MO = config_ATF
    elif model_type == 'GIN':         con_MO = config_GIN
    elif model_type == 'RNN':         con_MO = config_RNN
    else: print('Error !{MLP, AttentiveFP, GIN, RNN}'); return None
    con_MO['config_path'] = con_MO['model_path'].split('.')[0] + '.yml'
    # different weight of task, initial weight the same
    con_MO['weight_loss'] = [float(1.0/len(names))] * len(names)
    con_MO['MAX_EPOCH'] = MAX_EPOCH
    return con_MO



In [63]:
yml_name = 'Lipophilicity_AstraZeneca_scale/AttentiveFP_ST_1.yml'
p = yml_report(yml_name, ver=False)
print('Evaluate all metrics, just set metrics_dict as {}')
eval_perf_list(p, 'Lipophilicity_AstraZeneca', metrics_dict={})

print(f'Evaluate selected metrics {d}')
eval_perf_list(p, 'Lipophilicity_AstraZeneca', metrics_dict=d)


Evaluate all metrics, just set metrics_dict as {}
******************** Lipophilicity_AstraZeneca ******************** 
		 mae 	 	 mse 	 	 rmse 	 	 r2 	 
	| 0.353 +/- 0.000 | 0.239 +/- 0.000 | 0.489 +/- 0.000 | 0.834 +/- 0.000 | 

Evaluate selected metrics {'reg': [0, 2, 3], 'cls': [0, 5, 6]}
******************** Lipophilicity_AstraZeneca ******************** 
		 mae 	 	 rmse 	 	 r2 	 
	| 0.353 +/- 0.000 | 0.489 +/- 0.000 | 0.834 +/- 0.000 | 



In [64]:
yml_name = '/content/drive/MyDrive/ADMET/M5/MLP_MT_1.yml'
p = yml_report(yml_name, ver=False)
name = ['CYP2C19_Veith', 'CYP2D6_Veith', 'CYP3A4_Veith',
                         'CYP1A2_Veith', 'CYP2C9_Veith']

eval_perf_list(p, name)

******************** CYP2C19_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.776 +/- 0.000 | 0.762 +/- 0.000 | 0.847 +/- 0.000 | 

******************** CYP2D6_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.843 +/- 0.000 | 0.315 +/- 0.000 | 0.815 +/- 0.000 | 

******************** CYP3A4_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.726 +/- 0.000 | 0.667 +/- 0.000 | 0.816 +/- 0.000 | 

******************** CYP1A2_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.816 +/- 0.000 | 0.808 +/- 0.000 | 0.900 +/- 0.000 | 

******************** CYP2C9_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.795 +/- 0.000 | 0.666 +/- 0.000 | 0.862 +/- 0.000 | 



#### Test single train scaling effects

In [55]:
def eval_multi_run(model_type, name, folder_list, repeat_time=3, run_type=None):
    if isinstance(name, str): name = [name]
    if run_type == None: run_type = 'ST' if len(name) == 1 else 'MT'
    if isinstance(folder_list, str): folder_list = [folder_list]

    for folder_ in folder_list:
        print('checking', folder_)
        try:
            perform_list = []
            for i in range(repeat_time):
                yml_name = f'{folder_}/{model_type}_{run_type}_{i}.yml'
                try:
                    p = yml_report(yml_name, ver=False); perform_list.append(p)
                except:
                    print('cannot read', yml_name); break
            if len(perform_list) != 0: eval_perf_list(perform_list, name)
        except: pass
    print()

In [57]:
name = 'Lipophilicity_AstraZeneca'
folder_list = [f'{name}_scale']
for model_type in model_types:
    print(model_type)
    eval_multi_run(model_type, name, folder_list, run_type='ST')

MLP
checking Lipophilicity_AstraZeneca_scale
repeated num # 3
******************** Lipophilicity_AstraZeneca ******************** 
		 mae 	 	 rmse 	 	 r2 	 
	| 0.663 +/- 0.011 | 0.864 +/- 0.007 | 0.495 +/- 0.008 | 
 idx 0: | 0.654 +/- 0.000 | 0.855 +/- 0.000 | 0.506 +/- 0.000 | 


AttentiveFP
checking Lipophilicity_AstraZeneca_scale
repeated num # 3
******************** Lipophilicity_AstraZeneca ******************** 
		 mae 	 	 rmse 	 	 r2 	 
	| 0.365 +/- 0.009 | 0.503 +/- 0.011 | 0.824 +/- 0.007 | 
 idx 1: | 0.353 +/- 0.000 | 0.489 +/- 0.000 | 0.834 +/- 0.000 | 


GIN
checking Lipophilicity_AstraZeneca_scale
repeated num # 3
******************** Lipophilicity_AstraZeneca ******************** 
		 mae 	 	 rmse 	 	 r2 	 
	| 0.522 +/- 0.028 | 0.683 +/- 0.030 | 0.684 +/- 0.028 | 
 idx 0: | 0.498 +/- 0.000 | 0.660 +/- 0.000 | 0.706 +/- 0.000 | 


RNN
checking Lipophilicity_AstraZeneca_scale
repeated num # 3
******************** Lipophilicity_AstraZeneca ******************** 
		 mae 	 	 rmse

##### Test single task scaling effect

In [58]:
names_reg

['Caco2_Wang',
 'Lipophilicity_AstraZeneca',
 'HydrationFreeEnergy_FreeSolv',
 'Solubility_AqSolDB',
 'LD50_Zhu']

In [ ]:
repeat_time = 3
for name in names_reg[:1]:
    run_type = 'ST'
    trn, val, tst = collect_data_10_27([name])
    for scale_task in [False, True]:
        if scale_task == False: folder_name = f'{name}_no_scale'
        else: folder_name = f'{name}_scale'
        make_path(folder_name)
        print('\n\n\n\n\ninfo will be saved at', folder_name)
        trn, val, tst, dict_scale = scale(trn, val, tst, scale_task=scale_task)

        for model_type in model_types:
            print(model_type)
            config = get_config(model_type, [name])
            config['scale_dict'] = dict_scale
            trn_l, val_l, tst_l, vocab = get_multi_loader(trn, val, tst, config)
            if vocab != None and config['vocab'] == None:
                config['vocab'] = vocab # update config vocab info
                print('RNN, no vocab, use data update | vocab len:', len(vocab))

            perfs = [] # a list to store the performance outputted from the model
            for i in range(repeat_time):
                print(f'\nRun # {i} for {model_type} {run_type}', end='\t')
                save_dir = f'{folder_name}/{model_type}_{run_type}_{i}'
                config['model_path']  = save_dir + '.pt'
                config['config_path'] = save_dir + '.yml'

                try: # try open yml file, if file exists, and no need train
                    with open(config['config_path'], 'r') as f:
                        data = yaml.safe_load(f)
                    if data != None:
                        p = yml_report(data); print(' pre data loaded \n')
                except:  # model was not trained yet, train the model
                    models = PRED(**config)
                    p = models.train(trn_l, val_l, tst_l)
                perfs.append(p)
            eval_perf_list(perfs, name)

Epoch:15 [Train] Loss: 5.718 | [Valid] Loss: 2.487	SAVE MODEL: loss: 2.574 -> 2.487 | runtime: 2.499 ms
Epoch:16 [Train] Loss: 4.879 | [Valid] Loss: 1.979	SAVE MODEL: loss: 2.487 -> 1.979 | runtime: 2.438 ms
Epoch:17 [Train] Loss: 5.021 | [Valid] Loss: 1.671	SAVE MODEL: loss: 1.979 -> 1.671 | runtime: 2.435 ms
Epoch:18 [Train] Loss: 4.772 | [Valid] Loss: 1.473	SAVE MODEL: loss: 1.671 -> 1.473 | runtime: 2.551 ms
Epoch:19 [Train] Loss: 4.487 | [Valid] Loss: 1.407	SAVE MODEL: loss: 1.473 -> 1.407 | runtime: 2.483 ms
Epoch:20 [Train] Loss: 3.966 | [Valid] Loss: 1.203	SAVE MODEL: loss: 1.407 -> 1.203 | runtime: 2.565 ms
Epoch:21 [Train] Loss: 4.344 | [Valid] Loss: 0.924	SAVE MODEL: loss: 1.203 -> 0.924 | runtime: 2.492 ms


In [ ]:
repeat_time = 3
for name in names_reg[3:]:
    run_type = 'ST'
    trn, val, tst = collect_data_10_27([name])
    for scale_task in [False, True]:
        if scale_task == False: folder_name = f'{name}_no_scale'
        else: folder_name = f'{name}_scale'
        make_path(folder_name)
        print('\n\n\n\n\ninfo will be saved at', folder_name)
        trn, val, tst, dict_scale = scale(trn, val, tst, scale_task=scale_task)

        for model_type in model_types:
            print(model_type)
            config = get_config(model_type, [name])
            config['scale_dict'] = dict_scale
            trn_l, val_l, tst_l, vocab = get_multi_loader(trn, val, tst, config)
            if vocab != None and config['vocab'] == None:
                config['vocab'] = vocab # update config vocab info
                print('RNN, no vocab, use data update | vocab len:', len(vocab))

            perfs = [] # a list to store the performance outputted from the model
            for i in range(repeat_time):
                print(f'\nRun # {i} for {model_type} {run_type}', end='\t')
                save_dir = f'{folder_name}/{model_type}_{run_type}_{i}'
                config['model_path']  = save_dir + '.pt'
                config['config_path'] = save_dir + '.yml'

                try: # try open yml file, if file exists, and no need train
                    with open(config['config_path'], 'r') as f:
                        data = yaml.safe_load(f)
                    if data != None:
                        p = yml_report(data); print(' pre data loaded \n')
                except:  # model was not trained yet, train the model
                    models = PRED(**config)
                    p = models.train(trn_l, val_l, tst_l)
                perfs.append(p)
            eval_perf_list(perfs, name)

In [26]:
eval_perf_list([p], name[:2], metrics_dict=d)

repeated num # 1
******************** CYP2C19_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.776 +/- 0.000 | 0.762 +/- 0.000 | 0.847 +/- 0.000 | 


repeated num # 1
******************** CYP2D6_Veith ******************** 
		 acc 	 	 f1 	 	 auc 	 
	| 0.843 +/- 0.000 | 0.315 +/- 0.000 | 0.815 +/- 0.000 | 


